# Set up environment

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from __future__ import absolute_import
from __future__ import with_statement
from __future__ import division
from __future__ import nested_scopes
from __future__ import generators
from __future__ import unicode_literals
from __future__ import print_function

# Load scipy/numpy/matplotlib
from   scipy.linalg import expm
import matplotlib.pyplot as plt
from   pylab import *
import numpy as np
import scipy.io
from warnings import warn

# Configure figure resolution
plt.rcParams['figure.figsize'] = (12.0, 6.0)
plt.rcParams['savefig.dpi'   ] = 100

from izh       import * # Routines for sampling Izhikevich neurons
from plot      import * # Misc. plotting routines
from glm       import * # GLM fitting
from arppglm   import * # Sampling and integration
from utilities import * # Other utilities
from arguments import * # Argument verification

import os
dtype='float64'
flags = 'mode=FAST_RUN,device=gpu,floatX=%s'%dtype
if dtype!='float64':
    flags += ',warn_float64=warn'
os.environ["THEANO_FLAGS"] = flags

import theano
import theano.tensor as T

from theano_arppglm import *

print('Workspace Initialized')

# Load saved GLM features

In [ ]:
#filename = 'saved_training_model.mat'
filename = 'saved_training_model_badburster.mat'

saved_training_model = scipy.io.loadmat(filename)
K  = np.array(saved_training_model['K'],dtype=dtype)
B  = np.array(saved_training_model['B'],dtype=dtype)
By = np.array(saved_training_model['By'],dtype=dtype)
Bh = np.array(saved_training_model['Bh'],dtype=dtype)
A  = np.array(saved_training_model['A'],dtype=dtype)
C  = np.array(saved_training_model['C'],dtype=dtype)
Y  = asvector(np.array(saved_training_model['Y'],dtype=dtype))
dt = np.array(saved_training_model['dt'],dtype=dtype)

Bh_train = saved_training_model['Bh_train']
By_train = saved_training_model['By_train']
X_train  = concatenate([By_train,Bh_train],axis=1)
Y_train  = asvector(saved_training_model['Y_train'])

Bh_test  = saved_training_model['Bh_test']
By_test  = saved_training_model['By_test']
X_test   = concatenate([By_test,Bh_test],axis=1)
Y_test   = asvector(saved_training_model['Y_test'])
 
K  = int(scalar(K))
N  = prod(Y.shape)

'''# Don't use all training data
STARTPLOT = 0#2000
NPLOT = N#5000
Y  = Y[STARTPLOT:STARTPLOT+NPLOT]
By = By[STARTPLOT:STARTPLOT+NPLOT]
Bh = Bh[STARTPLOT:STARTPLOT+NPLOT]
X  = X[STARTPLOT:STARTPLOT+NPLOT]
'''
N = len(X_train)
STARTPLOT=0
NPLOT=N

print('Saved GLM features loaded')
print(N)

### GLM helpers

In [ ]:
#STARTSHOW = 14000
#STOPSHOW = 16000
STARTSHOW = 0
STOPSHOW = N

In [ ]:
def lograte(Bh,By,p):
    '''
    Log-intensity of point process model on this dataset
    Predicted using the standard GLM way
    '''
    m       = array(p).ravel()[0]
    beta    = ascolumn(p[1:K+1])
    beta_st = ascolumn(p[1+K:])
    lograte = m + Bh.dot(beta_st) + By.dot(beta)
    return lograte

def logmean(Bh,M1,p):
    '''
    Projected history process
    Predicted using history-process means
    '''
    m       = array(p).ravel()[0]
    beta    = ascolumn(p[1:K+1])
    beta_st = ascolumn(p[1+K:])
    M1      = np.squeeze(M1)
    return (beta.T.dot(M1.T))[0] + (m + Bh.dot(beta_st))[:,0]

def get_stim(Bh,p):
    m        = array(p).ravel()[0]
    beta     = ascolumn(p[1:K+1])
    beta_st  = ascolumn(p[1+K:])
    stim     = (m + Bh.dot(beta_st))[:,0]
    return stim

def filter_GLM_np(Bh,p):
    m        = array(p).ravel()[0]
    beta     = ascolumn(p[1:K+1])
    beta_st  = ascolumn(p[1+K:])
    stim     = get_stim(Bh,p)
    allM1_np = np.zeros((N,K))
    M1       = np.zeros((K,1))
    for i in range(N):
        R   = scalar(sexp(p0[1:K+1].dot(M1)+m+stim[i]))
        M1 += A.dot(M1)*dt + C.dot(R)
        allM1_np[i] = M1[:,0]
    return allM1_np

def addspikes_(Y_=None):
    if Y_ is None or Y_ is True:
        Y_ = Y
    for t in find(Y_>0):
        axvline(t,color=OCHRE,lw=0.4)
    
def niceaxis(plotspikes=True):
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore",message='No labelled objects found')
        legend()
    simpleraxis()
    xlim(STARTSHOW,STOPSHOW)
    if plotspikes is True or not plotspikes is None:
        addspikes_(plotspikes)

print('GLM helpers done')

In [ ]:
# Re-fit GLM
m,bhat  = fitGLM(X_train,asvector(Y_train))

# Re-pack model parameters
p0      = np.zeros((1+len(bhat)))
p0[0 ]  = m
p0[1:]  = bhat

allM1_np = filter_GLM_np(Bh_train,p0)
subplot(311)
plot(lograte(Bh_train,By_train,p0),lw=0.4,label='conditional intensity')
plot(logmean(Bh_train,allM1_np,p0),lw=0.4,label='mean-field',color=RUST)
niceaxis()
ylim(min(lograte(Bh_train,By_train,p0)),5)

# Filtering 

### Parameters at which to filter

In [ ]:
oversample = 2    # Integration resolution
maxrate    = 10.0 # Largest allowed rate
maxvcorr   = 10.0 # Largest allowed variance correction
dt         = 1.0  # Data time resolution
reg_cov    = 1e-4
reg_rate   = 1e-9

p = p0.copy()
#p[1:K+1] *= 0.775
stim_np = get_stim(Bh_train,p)
beta_np = ascolumn(p[1:K+1])
print('Filtering using p=',v2str(p))

# Helper function to compute negative expected log-likelihood
def post_hoc_nll(LR,LV):
    R0 = sexp(LR)
    R1 = R0*(1+0.5*LV)
    ELL  = np.mean(Y*LR - R1)
    return -ELL

### Build Theano routines

For integrating moments (not conditioned on data), filtering (conditioned on data), and filtering using surrogate likelihoods (Gaussian approximations).

In [ ]:
from theano_arppglm import *

GLM_log_intensity, GLMNLL_f, GLMNLL_g, GLMNLL_h = build_ML_GLM_likelihood_theano()

integrate_moments_theano, EMNLL_filt, EMNLL_grad = build_integrate_moments_theano(N,A,C,
    dt          = dt,
    oversample  = oversample,
    maxrate     = maxrate,
    maxvcorr    = maxvcorr,
    method      = "second_order",
    int_method  = "euler")

filter_moments_theano, NLL_filt, NLL_grad = build_filter_moments_theano(N,A,C,
    dt          = dt,
    oversample  = oversample,
    maxrate     = maxrate,
    maxvcorr    = maxvcorr,
    method      = "second_order",
    int_method  = "euler",
    measurement = "moment",
    reg_cov     = reg_cov,
    reg_rate    = reg_rate,
    return_surrogates = True)

filter_surrogate_theano, SNLL_filt, SNLL_grad = build_filter_moments_theano(N,A,C,
    dt          = dt,
    oversample  = oversample,
    maxrate     = maxrate,
    maxvcorr    = maxvcorr,
    method      = "second_order",
    int_method  = "euler",
    measurement = "moment",
    reg_cov     = reg_cov,
    reg_rate    = reg_rate,
    return_surrogates = False,
    use_surrogates    = True)

print('Theano functions bulit')

In [ ]:
p = p0.copy()
p[1:]*=0.25

m        = array(p).ravel()[0]
beta     = ascolumn(p[1:K+1])
beta_st  = ascolumn(p[1+K:])
stim     = (m + Bh.dot(beta_st))[:,0]

tic()
allLRn,allLVn,allM1n,allM2n,nlln,mrn,vrn = filter_moments(stim,Y,A,beta,C,p[0],
    dt          = dt,
    oversample  = oversample,
    maxrate     = maxrate,
    maxvcorr    = maxvcorr,
    method      = "second_order",
    int_method  = "euler",
    measurement = "moment",
    reg_cov     = reg_cov,
    reg_rate    = reg_rate,
    return_surrogates = True)
toc()
subplot(411)
stderrplot(allLRn,allLVn,color=BLACK,lw=0.5)
niceaxis()
ylim(-15,5)
xlim(STARTSHOW,STOPSHOW)
title('Filtering, numpy')
print('nll, numpy',nlln)

tic()
allLRt,allLVt,allM1t,allM2t,nllt,mrt,vrt = filter_moments_theano(Bh,Y,p)
toc()
subplot(412)
stderrplot(allLRt,allLVt,color=BLACK,lw=0.5)
niceaxis()
ylim(-15,5)
xlim(STARTSHOW,STOPSHOW)
title('Filtering, theano')
print('nll, theano',nllt)

subplot(413)
plot(allLRn,color=BLACK,label='log-λ numpy')
plot(allLRt,':',color=RUST,label='log-λ theano')
plot(allLRni,color=AZURE,label='log-λ no measurements')
niceaxis()
xlim(STARTSHOW,STOPSHOW)
#xlim(14900,15100)

tight_layout()

### Check that Theano matches GLMfit

#### Optimization summary report helpers

In [ ]:
def get_sample(Bh,p,M):
    m        = array(p).ravel()[0]
    beta     = ascolumn(p[1:K+1])
    beta_st  = ascolumn(p[1+K:])
    stim     = (m + Bh.dot(beta_st))[:,0]
    return ensemble_sample(stim,B,beta,M=100)

def summarize_result(X,Y,Bh,result):
    Y = asvector(Y)
    
    # Sample from point-process models
    y1,l1 = get_sample(Bh,p0,1000)
    y2,l2 = get_sample(Bh,result,1000)
    print('True   mean rate is',mean(Y))
    print('GLMfit mean rate is',mean(y1))
    print('MCfilt mean rate is',mean(y2))
    print('GLMfit mean log-likelihood is',mean(Y[:,None]*l1 - sexp(l1)))
    print('MCfilt mean log-likelihood is',mean(Y[:,None]*l2 - sexp(l2)))
    # FIGURE
    figure(figsize=(10,8))
    
    subplot(411)
    LM,LV,_,_ = integrate_moments_theano(Bh,result)
    stderrplot(LM,LV,color=BLACK,lw=0.4)
    niceaxis(Y)
    title('Moment-closure density')
    
    subplot(412)
    LM,LV,_,_,NLLS,_,_ = filter_moments_theano(Bh,Y,result)
    stderrplot(LM,LV,color=BLACK,lw=0.4)
    niceaxis(Y)
    title('Conditional density')
    NSAMP = 100
    
    subplot(413)
    pcolormesh(-int32(y1[:,:NSAMP].T>0),cmap='gray')
    noaxis(); 
    xlabel('Time (ms)'); 
    ylabel('Sample',fontsize=9); 
    niceaxis(Y)
    title('GLMfit')
    
    subplot(414)
    pcolormesh(-int32(y2[:,:NSAMP].T>0),cmap='gray')
    noaxis(); 
    xlabel('Time (ms)'); 
    ylabel('Sample',fontsize=9); 
    niceaxis(Y)
    title('MCfilt')
    tight_layout()
    
    figure()
    p = array(result)
    m = p[0]
    b = p[1:K+1]
    h = p[K+1:]
    b.shape,B.shape
    
    subplot(211)
    l = X.dot(p[1:]) + p[0]
    plot(l,color=BLACK,lw=1,label='MCfilt')
    l = X.dot(p0[1:]) + p0[0]
    plot(l,color=RUST,lw=1,label='GLMfit')
    xlim(STARTSHOW,STOPSHOW)
    simpleraxis()

    subplot(212)
    dB = log(10)*10
    plot(b.dot(B)*dB)
    plot(h.dot(B)*dB)
    axhline(0,color='k',lw=0.5)
    simpleraxis()
    xlim(0,150)
    tight_layout()

    
print('done')

#### initial conditions and objective

In [ ]:
GLM_result = p0 + randn(*np.shape(p0))*10
GLM_result[1:]*=0.2
GLM_result = p0*0

large   = sqrt(np.finfo('float32').max)

X,Y = X_train,Y_train

def objective(p):
    NLL = GLMNLL_f(X,Y,p)[0]
    return NLL if isfinite(NLL) else large

def gradient(p):
    return GLMNLL_g(X,Y,p)[0]

def hessian(p):
    return GLMNLL_h(X,Y,p)[0]

def og(p):
    o = objective(p)
    g = gradient(p)
    return o,g

# Check gradients
g1 = gradient(GLM_result)
g2 = numeric_grad(objective,GLM_result,10**-4)
#print(g1)
#print(g2)
print('Mean absolute relative error in gradient:',mean(abs(g1-g2)/abs(g1)))

# check hessian
h1 = hessian(GLM_result)
h2 = numeric_hess(gradient,GLM_result,1e-4)
#print('Symbolic:\n','\n'.join(map(v2str,h1)))
#print('Numeric:\n','\n'.join(map(v2str,h2)))
#print('Difference:\n','\n'.join(map(v2str,h1-h2)))
print('Mean absolute relative error in hessian:',mean(abs(h1-h2)/abs(h1)))

In [ ]:
print('Starting optimization')
try:
    GLM_result = minimize_retry(og,GLM_result,jac=True,hess=hessian,verbose=True,options={'eps':1e-4},tol=1e-12)
    GLM_result = minimize_retry(og,GLM_result,jac=True,verbose=True,options={'eps':1e-4},tol=1e-12)
    GLM_result = minimize_retry(objective,GLM_result,jac=False,verbose=True,tol=1e-12,
                           options={'eps':1e-4,'maxiter':1e19,'maxfev':1e19})
    GLM_result = minimize_retry(objective,GLM_result,jac=False,verbose=True,simplex_only=True,tol=1e-12,
                           options={'maxiter':inf,'maxfev':inf})
    print("Finished optimization")
except KeyboardInterrupt:
    print('Optimization paused')

print('x=','['+','.join([np.float128(x).astype(str) for x in GLM_result])+']')
print("Total absolute change from GLM fit is",sum(abs(GLM_result-p0)))

summarize_result(X_train,asvector(Y_train),Bh_train,GLM_result)
#GLM_log_intensity, GLMNLL_f, GLMNLL_g

### Check moment integration

In [ ]:
tic()
allLRni,allLVni,allM1ni,allM2ni = integrate_moments(stim_np,A,beta_np,C,
    dt          = dt,
    oversample  = oversample,
    maxrate     = maxrate,
    maxvcorr    = maxvcorr,
    method      = "second_order",
    int_method  = "euler")
toc()
subplot(411)
stderrplot(allLRni,allLVni,color=BLACK,lw=0.5)
niceaxis()
ylim(-20,10)
xlim(STARTSHOW,STOPSHOW)
title('Integrating, numpy')
print('ENLL=',post_hoc_nll(allLRni,allLVni))

tic()
allLRti,allLVti,allM1ti,allM2ti = integrate_moments_theano(Bh,p0)
toc()
subplot(412)
stderrplot(allLRti,allLVti,color=BLACK,lw=0.5)
niceaxis()
ylim(-20,10)
xlim(STARTSHOW,STOPSHOW)
title('Integrating, theano')
print('ENLL=',post_hoc_nll(allLRti,allLVti))

subplot(413)
plot(allLRni,color=BLACK,label='log-λ, numpy')
plot(allLRti,color=RUST,label='log-λ, theano')
niceaxis()
xlim(STARTSHOW,STOPSHOW)
title('Log-intensity')

tight_layout()

# Optimize

Here goes nothing. We want to combine conditional NLL with expected NLL. Expected NLL fits based on single time marginals of model response to stimuli, not conditioned on observed spikes. NLL conditionde on observed spikes, but may be stable even for unstable models and so is not ideal.

### initial conditions

In [ ]:
p1     = p0.copy()
ELL_result = p1

ELL_result = [-5.14182409466,1.11109383391,-4.82878604739,1.83673294557,1.79422798288,-3.48898983213,2.98998295676,-2.43428255153,0.71238421449,1.37600129637,-1.72296587502,2.17808722548,-1.31781555208,0.85589939253,-0.480755976108,0.24930797051,-0.0928260843248]
ELL_result = [-5.3278358189,0.689932896008,-4.36386887719,2.59152547217,0.0190567168914,-2.27453018523,2.62021528673,-2.49639075347,0.901691011863,1.51751189519,-2.00478119387,2.38041164695,-1.35729594407,0.850552046267,-0.467364999388,0.238760435641,-0.0927765029538]
ELL_result = [-5.10885924759,1.12824061962,-2.41714077891,1.01788191554,-0.00115025957431,-1.39257503717,1.21171230522,-1.6188025382,0.401108538778,0.507814976896,-0.987658945039,1.73799933077,-1.01917944975,0.65723363294,-0.34806237986,0.191621899837,-0.0747745444539]

print('done')

## Optimize only expected Log-likelihood

In [ ]:
large   = sqrt(np.finfo('float32').max)

def objective(p):
    # Expected negative log-likelihood
    NLL = EMNLL_filt(Bh,Y,p)[0]
    if not isfinite(NLL):
        NLL = large
    return NLL

def gradient(p):
    GNLL = EMNLL_grad(Bh,Y,p)[0]
    return GNLL

def og(p):
    o = objective(p)
    g = gradient(p)
    return o,g

print('Starting optimization')

try:
    ELL_result = minimize_retry(og,ELL_result,jac=True,verbose=True)
    ELL_result = minimize_retry(objective,ELL_result,jac=False,verbose=True,simplex_only=True)
    print("Finished optimization")
except KeyboardInterrupt:
    print('Optimization paused')
    
print('x=','['+','.join([np.float128(x).astype(str) for x in ELL_result])+']')
print("Total absolute change from GLM fit is",sum(abs(ELL_result-p0)))

summarize_result(X_train,Y_train,Bh_train,ELL_result)
summarize_result(X_test,Y_test,Bh_test,ELL_result)

# Incorporate filtering (conditional) likelihood

In [ ]:
p = ELL_result.copy()

### Check filtering

In [ ]:
p = p0

In [ ]:
m        = array(p).ravel()[0]
beta     = ascolumn(p[1:K+1])
beta_st  = ascolumn(p[1+K:])
stim     = (m + Bh.dot(beta_st))[:,0]

tic()
allLRn,allLVn,allM1n,allM2n,nlln,mrn,vrn = filter_moments(stim,Y,A,beta,C,p[0],
    dt          = dt,
    oversample  = oversample,
    maxrate     = maxrate,
    maxvcorr    = maxvcorr,
    method      = "second_order",
    int_method  = "euler",
    measurement = "moment",
    reg_cov     = reg_cov,
    reg_rate    = reg_rate,
    return_surrogates = True)
toc()
subplot(411)
stderrplot(allLRn,allLVn,color=BLACK,lw=0.5)
niceaxis()
ylim(-15,5)
xlim(STARTSHOW,STOPSHOW)
title('Filtering, numpy')
print('nll, numpy',nlln)

tic()
allLRt,allLVt,allM1t,allM2t,nllt,mrt,vrt = filter_moments_theano(Bh,Y,p)
toc()
subplot(412)
stderrplot(allLRt,allLVt,color=BLACK,lw=0.5)
niceaxis()
ylim(-15,5)
xlim(STARTSHOW,STOPSHOW)
title('Filtering, theano')
print('nll, theano',nllt)

subplot(413)
plot(allLRn,color=BLACK,label='log-λ numpy')
plot(allLRt,':',color=RUST,label='log-λ theano')
plot(allLRni,color=AZURE,label='log-λ no measurements')
niceaxis()
xlim(STARTSHOW,STOPSHOW)
#xlim(14900,15100)

tight_layout()

### Check filtering using surrogate measurements

In [ ]:
tic()
allLRns,allLVns,allM1ns,allM2ns,nllns = filter_moments(stim,Y,A,beta,C,p[0],
    dt          = dt,
    oversample  = oversample,
    maxrate     = maxrate,
    maxvcorr    = maxvcorr,
    method      = "second_order",
    int_method  = "euler",
    measurement = "moment",
    reg_cov     = reg_cov,
    reg_rate    = reg_rate,
    return_surrogates = False,
    use_surrogates = (mrn,vrn))
toc()
subplot(411)
stderrplot(allLRns,allLVns,color=BLACK,lw=0.5)
niceaxis()
ylim(-20,10)
xlim(STARTSHOW,STOPSHOW)
title('Filtering, numpy')
print('nll, surrogate numpy',nllns)

tic()
allLRts,allLVts,allM1ts,allM2ts,nllts = filter_surrogate_theano(Bh,Y,p,mrt,vrt)
toc()
subplot(412)
stderrplot(allLRt,allLVt,color=BLACK,lw=0.5)
niceaxis()
ylim(-20,10)
xlim(STARTSHOW,STOPSHOW)
title('Filtering, theano')
print('nll, surrogate theano',nllts)

subplot(413)
plot(mrn,color=BLACK,label='Surrogate mean updates, numpy')
plot(mrt,color=RUST,label='Surrogate mean updates, theano')
niceaxis()
title('Surrogate means')
xlim(STARTSHOW,STOPSHOW)

subplot(414)
plot(allLRn,color=BLACK,label='Conditional log-λ, numpy')
plot(allLRt,color=RUST,label='Conditional log-λ, theano')
plot(allLRni,color=AZURE,label='Moment closure mean')
plot(allLRns,color=CRIMSON,label='Surrogate log-λ, numpy')
plot(allLRts,':',color=MAUVE,zorder=inf,label='Surrogate log-λ, theano')
niceaxis()
title('Using surrogate updates')
xlim(STARTSHOW,STOPSHOW)

tight_layout()

## Optimize using ELL and conditional hybrid

In [ ]:
#resut = (result+p0)*0.5
#result = p0
result = ELL_result#(ELL_result+GLM_result)*0.5

In [ ]:
strict  = False
verbose = False
large   = sqrt(np.finfo('float32').max)

allLRt,allLVt,allM1t,allM2t,nllt,mrt,vrt = filter_moments_theano(Bh,Y,result)

def objective(p):
    NLL = SNLL_filt(Bh,Y,p,mrt,vrt)[0]
    if not isfinite(NLL):
        NLL = large
    return NLL

p = result
print(SNLL_grad(Bh,Y,p,mrt,vrt)[0])
from glm import numeric_grad
print(numeric_grad(objective,p,1e-3))

In [ ]:
x= [-5.21847116314,0.454763857218,-0.726327441249,0.738281986681,-0.626254125845,0.494580375459,-0.327687909616,0.150363362699,-0.0282694097873,-4.71687411694,2.93672402134,0.294688287784,-0.536458963393,0.545767817347,-0.517988679391,0.430871438156,-0.320890257658]

In [ ]:
result = ELL_result*0.5 + p*0.5

In [ ]:
def objective(p):
    NLLM = SNLL_filt(Bh,Y,p,mrt,vrt)[0]
    #NLLP = GLMNLL_f(X,Y,p)[0]
    #NLLQ = EMNLL_filt(X,Y,p)
    NLL = NLLM #+ NLLP
    #NLL -= 0.0880704571813
    #NLL -= -2.39810128699e-09
    #NLL -= -1.37653780625e-13 
    return NLL if isfinite(NLL) else large

def gradient(p):
    GNLLM = SNLL_grad(Bh,Y,p,mrt,vrt)[0]
    #GNLLP = GLMNLL_g(X,Y,p)[0]
    #GEML = 
    #GENL = EMNLL_grad(Bh,Y,p)[0]
    g = GNLLM #+ GNLLP
    return g

def og(p):
    o = objective(p)
    g = gradient(p)
    return o,g
    
try:
    #result = minimize_retry(og,result,jac=True,verbose=verbose,simplex_only=False,options={'eps':1e-3})
    #result = minimize_retry(objective,result,jac=False,verbose=verbose,simplex_only=False,options={'eps':1e-3})
    result = minimize_retry(objective,result,jac=False,verbose=verbose,simplex_only=True)
    print("Finished optimization")
except KeyboardInterrupt:
    print('Optimization paused')
    
print('x=','['+','.join([np.float128(x).astype(str) for x in result])+']')
print("Total absolute change from GLM fit is",sum(abs(result-p0)))

summarize_result(X_train,asvector(Y_train),Bh_train,result)
summarize_result(X_test,asvector(Y_test),Bh_test,result)

In [ ]:

summarize_result(X_train,asvector(Y_train),Bh_train,result)
summarize_result(X_test,asvector(Y_test),Bh_test,result)